In [2]:
import numpy as np
import cv2
import os
import math

In [7]:
# Configuration
IMG_DIR = "../data/val/normal/"
OUT_DIR = "../data/cropped_val/normal"
NEW_ROW = 150
NEW_COL = 130
EYE_DISTANCE = 70.0
EYEROW = 45.0

In [31]:
def rotate_image(img, x1, y1, x2, y2):
    angle = -math.degrees(math.atan2(y2 - y1, x2 - x1))
    center = ((x1 + x2) // 2, (y1 + y2) // 2)
    rot_mat = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(img, rot_mat, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)
    new_eye1 = np.dot(rot_mat, np.array([x1, y1, 1]))
    new_eye2 = np.dot(rot_mat, np.array([x2, y2, 1]))
    return rotated, new_eye1.astype(int), new_eye2.astype(int)

def resize_image(img, eye1, eye2):
    current_dist = np.linalg.norm(eye2 - eye1)
    scale = EYE_DISTANCE / current_dist
    new_size = (int(img.shape[1] * scale), int(img.shape[0] * scale))
    resized = cv2.resize(img, new_size, interpolation=cv2.INTER_LINEAR)
    return resized, scale

def crop_image(img, eye_left, eye_right, scale):
    left_eye_x = eye_left[0] * scale
    right_eye_x = eye_right[0] * scale
    eye_y = eye_left[1] * scale

    lft_col = int(left_eye_x - (NEW_COL - EYE_DISTANCE) / 2)
    upr_row = int(eye_y - EYEROW)
    rgt_col = lft_col + NEW_COL
    btm_row = upr_row + NEW_ROW

    # Ensure crop boundaries are within image
    h, w = img.shape[:2]
    lft_col = max(0, min(lft_col, w - NEW_COL))
    upr_row = max(0, min(upr_row, h - NEW_ROW))

    cropped = img[upr_row:upr_row + NEW_ROW, lft_col:lft_col + NEW_COL]
    return cropped

def process_images(coord_file):
    with open(coord_file, 'r') as f:
        for line in f:
            tokens = line.strip().split()
            if len(tokens) != 5:
                continue
            filename, x1, y1, x2, y2 = tokens
            x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))

            filepath = os.path.join(IMG_DIR, filename)
            img = cv2.imread(filepath, cv2.IMREAD_COLOR)  # Read as color
            if img is None:
                print(f"Failed to load image: {filepath}")
                continue

            rotated, eye1_new, eye2_new = rotate_image(img, x1, y1, x2, y2)
            resized, scale = resize_image(rotated, eye1_new, eye2_new)
            cropped = crop_image(resized, eye1_new, eye2_new, scale)
            
            h, w = cropped.shape[:2]
            if (h, w) != (NEW_ROW, NEW_COL):
                print(f"Resizing {filename} from ({w}x{h}) to ({NEW_COL}x{NEW_ROW})")
                cropped = cv2.resize(cropped, (NEW_COL, NEW_ROW), interpolation=cv2.INTER_LINEAR)

            out_path = os.path.join(OUT_DIR, filename + ".crop.png")
            os.makedirs(OUT_DIR, exist_ok=True)
            cv2.imwrite(out_path, cropped)

In [ ]:
process_images('../data/val/normal/coord.dat')

In [8]:
FOLDER_PATH = "../data/cropped_test/rosacea"
i = 0

for filename in os.listdir(FOLDER_PATH):
    i+=1
    if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    filepath = os.path.join(FOLDER_PATH, filename)
    img = cv2.imread(filepath)

    if img is None:
        print(f"Failed to load {filename}")
        continue

    h, w = img.shape[:2]
    if (h, w) != (NEW_ROW, NEW_COL):
        print(f"Resizing {filename} from ({w}x{h}) to ({NEW_COL}x{NEW_ROW})")
        img = cv2.resize(img, (NEW_COL, NEW_ROW), interpolation=cv2.INTER_LINEAR)
        cv2.imwrite(filepath, img)
